In [1]:
import numpy as np
import pandas as pd
import pycountry
import Geohash

# TASK 1

In [42]:
city_file = '../../downloads/countries/worldcitiespop.csv'
city_names = pd.read_csv(city_file, dtype={
    'Country': str, 'City': str, 'AccentCity': str, 'Region': object, 
    'Population': np.float64,
    'Latitude': np.float64, 'Longitude': np.float64
})

In [43]:
city_names['ID'] = [Geohash.encode(k, v) for k, v in zip(
    city_names['Latitude'], city_names['Longitude'])]

In [44]:
cities = city_names.drop_duplicates(subset='ID', keep='first')

In [45]:
c_codes = cities['Country'].str.upper()
cities = cities.assign(Country=c_codes)

In [46]:
cities.head()

,Country,City,AccentCity,Region,Population,Latitude,Longitude,ID
0,AD,aixas,Aixàs,06,NaN,42.483333,1.466667,sp919mggrg3f
1,AD,aixirivali,Aixirivali,06,NaN,42.466667,1.500000,sp91d4u6283n
4,AD,aixovall,Aixovall,06,NaN,42.466667,1.483333,sp919fb4k0mq
5,AD,andorra,Andorra,07,NaN,42.500000,1.516667,sp91f8bkkyhx
11,AD,ansalonga,Ansalonga,04,NaN,42.566667,1.516667,sp944t02ucu9


# TASK 2

In [72]:
stats_files = {
    2015: '../../downloads/countries/2015.csv',
    2016: '../../downloads/countries/2015.csv',
    2017: '../../downloads/countries/2015.csv',
}
dfs = []

for year, file in stats_files.items():
    df = pd.read_csv(file)
    df['Year'] = year
    dfs.append(df)

In [73]:
stats = pd.concat(dfs)

In [74]:
def get_iso(country):
    try:
        c = pycountry.countries.get(name=country)
        iso = c.alpha_2
    except KeyError:
        iso = None
    return iso

In [75]:
codes = [get_iso(x) for x in stats['Country']]
stats['ISO'] = codes

In [ ]:
stats.head()

# TASK 3

## DB Schema

<code>country(**iso2**, iso3, name, numeric, official_name)</code>

<code>subdiv(**code**, *country_code*, name, *parent*, type)</code>

<code>city(**id**, *country*, city, complete_name, region, population, latitude, longitude)</code>

<code>stats(**country, year**, country_name, region, happiness_rank, happiness, std_errror, gdp_pc, family, health, freedom, trust, generosity, dystopia)</code>


### SQL

<pre><code>
CREATE TABLE countries.country (
  iso2 CHAR(2) NOT NULL,
  iso3 CHAR(3) NULL,
  name VARCHAR(45) NOT NULL,
  numeric INT NULL,
  official_name VARCHAR(200) NULL,
  PRIMARY KEY (iso2),
  UNIQUE (iso3)
);

CREATE TABLE countries.subdiv (
  code VARCHAR(8) NOT NULL,
  country_code CHAR(2) NOT NULL,
  name VARCHAR(200) NOT NULL,
  parent VARCHAR(8) NULL,
  type VARCHAR(45) NULL,
  PRIMARY KEY (code),
  CONSTRAINT country_code_fk
    FOREIGN KEY (country_code)
    REFERENCES countries.country (iso2)
    ON DELETE CASCADE
    ON UPDATE CASCADE
);

ALTER TABLE countries.subdiv 
ADD CONSTRAINT parent_fk
  FOREIGN KEY (parent)
  REFERENCES countries.subdiv (code)
  ON DELETE SET NULL
  ON UPDATE CASCADE;
  
CREATE TABLE countries.city (
  id CHAR(12) NOT NULL,
  country CHAR(2) NULL,
  city_name VARCHAR(200) NOT NULL,
  complete_name VARCHAR(200) NULL,
  region CHAR(2) NULL,
  population FLOAT NULL,
  lat FLOAT NULL,
  lon FLOAT NULL,
  PRIMARY KEY (id),
  CONSTRAINT country_fk
    FOREIGN KEY (country)
    REFERENCES countries.country (iso2)
    ON DELETE SET NULL
    ON UPDATE CASCADE
);

CREATE TABLE countries.stats (
  country CHAR(2) NOT NULL,
  year INT NOT NULL,
  country_name VARCHAR(200) NULL,
  region VARCHAR(200) NULL,
  happiness_rank INT NULL,
  happiness FLOAT NULL,
  std_error FLOAT NULL,
  gdp_pc FLOAT NULL,
  family FLOAT NULL,
  health FLOAT NULL,
  freedom FLOAT NULL,
  trust FLOAT NULL,
  generosity FLOAT NULL,
  dystopia FLOAT NULL,
  PRIMARY KEY (country, year),
  CONSTRAINT country_fk
    FOREIGN KEY (country)
    REFERENCES countries.country (iso2)
    ON DELETE CASCADE
    ON UPDATE CASCADE
);
      
</code></pre>



## DB population
We will use pandas for interacting with the DB (the example is based on PostgreSQL).

<code>pip install psycopg2</code>

In [2]:
from pandas.io import sql
import psycopg2

In [14]:
class DB(object):
    
    def __init__(self, host, user, password, db, schema=None, charset='utf8mb4'):
        self.host = host
        self.user = user
        self.password = password
        self.db = db
        self.charset = charset
        self.schema = schema
    
    def connect(self):
        return psycopg2.connect(
            "dbname='{}' user='{}' host='{}' password='{}'".format(
                self.db, self.user, self.host, self.password
            ))
    
    @staticmethod
    def as_params(df, attribute_map=None):
        tuples, attributes = [], [attribute_map[x] if 
                                  attribute_map is not None else x for x in df.columns]
        for i, row in df.iterrows():
            v = [v if not pd.isnull(v) else None for v in row.values]
            tuples.append(tuple(v))
        return attributes, tuples 
    
    def insert(self, table, df, attribute_map=None):
        if self.schema is not None:
            table = ".".join([self.schema, table])
        attributes, tuples = DB.as_params(df, attribute_map=attribute_map)
        colstr = ", ".join(["{}".format(a) for a in attributes])
        valstr = ", ".join(["%s" for a in attributes])
        sql = "INSERT INTO " + table + " (" + colstr + ") VALUES (" + valstr + ");"
        conn = self.connect()
        cur = conn.cursor()
        cur.executemany(sql, tuples)
        conn.commit()
        cur.close()
        conn.close()

### Create a dataframe for pycountries data

In [4]:
countries = []
for country in pycountry.countries:
    countries.append(country.__dict__['_fields'])
cdf = pd.DataFrame(countries)

In [5]:
cdf.head()

,alpha_2,alpha_3,common_name,name,numeric,official_name
0,AW,ABW,NaN,Aruba,533,NaN
1,AF,AFG,NaN,Afghanistan,004,Islamic Republic of Afghanistan
2,AO,AGO,NaN,Angola,024,Republic of Angola
3,AI,AIA,NaN,Anguilla,660,NaN
4,AX,ALA,NaN,Åland Islands,248,NaN


In [15]:
db = DB(host='localhost', user='postgres', password='flintpsql', db='bdlab', 
        schema='countries')
a_map = {'alpha_2': 'iso2', 'alpha_3': 'iso3', 'name': 'name', 
         'numeric': 'numeric', 'official_name': 'official_name'}

In [16]:
db.insert('country', cdf[cdf.columns.difference(['common_name'])], attribute_map=a_map)

### pycountries subdivs

In [41]:
s_map = {'code': 'code', 'country_code': 'country_code', 'name': 'name', 
         'parent_code': 'parent', 'type': 'type'}
for i, country in cdf.iterrows():
    s = pycountry.subdivisions.get(country_code=country.alpha_2)
    subdivs = [x.__dict__['_fields'] for x in s]
    if len(subdivs) > 0:
        sdf = pd.DataFrame(subdivs)
        try:
            db.insert('subdiv', 
                      sdf[sdf.columns.difference(['parent'])].sort_values(
                          'parent_code', na_position='first'), 
                      attribute_map=s_map)
        except psycopg2.DataError:
            print (sdf.code.unique())
            print (sdf.parent_code.unique())
        except KeyError:
            print (sdf.columns)

### Cities
Use the code for TASK 1 above

In [47]:
cities.head()

,Country,City,AccentCity,Region,Population,Latitude,Longitude,ID
0,AD,aixas,Aixàs,06,NaN,42.483333,1.466667,sp919mggrg3f
1,AD,aixirivali,Aixirivali,06,NaN,42.466667,1.500000,sp91d4u6283n
4,AD,aixovall,Aixovall,06,NaN,42.466667,1.483333,sp919fb4k0mq
5,AD,andorra,Andorra,07,NaN,42.500000,1.516667,sp91f8bkkyhx
11,AD,ansalonga,Ansalonga,04,NaN,42.566667,1.516667,sp944t02ucu9


In [48]:
y_map = {'Country': 'country', 'City': 'city_name', 'AccentCity': 'complete_name', 
         'Region': 'region', 'Population': 'population', 'Latitude': 'lat',
        'Longitude': 'lon', 'ID': 'id'}

<code>cities</code> is a large dataframe. We would like to partitioning it before inserting.
Skip also countries not in the country table.

In [70]:
i, chunk = 0, 100000
chunks = []
cni = cdf.alpha_2.unique()
while chunk < cities.shape[0]:
    df = cities.iloc[i:chunk]
    df1 = df[df.Country.isin(cni)]
    df2 = df[~df.City.isnull()]
    chunks.append(df2)
    i = chunk
    chunk += 100000

In [71]:
for i, c in enumerate(chunks[6:]):
    print ('Inserting', i, 'of', len(chunks))
    db.insert('city', c, attribute_map=y_map)

Inserting 0 of 22
Inserting 1 of 22
Inserting 2 of 22
Inserting 3 of 22
Inserting 4 of 22
Inserting 5 of 22
Inserting 6 of 22
Inserting 7 of 22
Inserting 8 of 22
Inserting 9 of 22
Inserting 10 of 22
Inserting 11 of 22
Inserting 12 of 22
Inserting 13 of 22
Inserting 14 of 22
Inserting 15 of 22


### Stats
Use the code for TASK 2 above

In [77]:
t_map = {'ISO': 'country', 'Year': 'year', 'Country': 'country_name', 
         'Region': 'region', 'Happiness Rank': 'happiness_rank', 
         'Happiness Score': 'happiness',
         'Standard Error': 'std_error', 'Economy (GDP per Capita)': 'gdp_pc', 
         'Family': 'family', 'Health (Life Expectancy)': 'health', 
         'Freedom': 'freedom', 'Trust (Government Corruption)': 'trust', 
         'Generosity': 'generosity', 'Dystopia Residual': 'dystopia'}

In [80]:
db.insert('stats', stats[~stats.ISO.isnull()], attribute_map=t_map)